In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import scrapy
import pandas as pd
import time

In [2]:
# TEST
# Fetch the webpage
response = requests.get('http://example.com')
html = response.content

In [3]:
# Parse 
soup = BeautifulSoup(html, 'html.parser')

In [4]:
# Extract data and put it in a DF
data = {'tag':[], 'content':[]}
for tag in soup.find_all(True):
    data['tag'].append(tag.name)
    data['content'].append(tag.text)

df = pd.DataFrame(data)
print(df.head())

     tag                                            content
0   html  \n\nExample Domain\n\n\n\n\n\n\n\nExample Doma...
1   head                         \nExample Domain\n\n\n\n\n
2  title                                     Example Domain
3   meta                                                   
4   meta                                                   


In [5]:
# PROJECT

In [6]:
# Get state urls:
def get_state_urls(base_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(base_url, headers=headers)

    if response.status_code != 200:
        print("Failed to retrieve the page. Status code:", response.status_code)
        return {}

    soup = BeautifulSoup(response.content, 'html.parser')
    state_urls = {}

    for link in soup.find_all('a'):
        state_name = link.text.strip()
        relative_url = link.get('href')

        if relative_url and 'church-in-' in relative_url:
            full_url = base_url + '/' + relative_url
            state_urls[state_name] = full_url

    return state_urls

In [7]:

base_url = 'https://www.usachurches.org'
states = get_state_urls(base_url)
print(states)

{'Alabama': 'https://www.usachurches.org/church-in-al.htm', 'Alaska': 'https://www.usachurches.org/church-in-ak.htm', 'Arizona': 'https://www.usachurches.org/church-in-az.htm', 'Arkansas': 'https://www.usachurches.org/church-in-ar.htm', 'California': 'https://www.usachurches.org/church-in-ca.htm', 'Colorado': 'https://www.usachurches.org/church-in-co.htm', 'Connecticut': 'https://www.usachurches.org/church-in-ct.htm', 'Delaware': 'https://www.usachurches.org/church-in-de.htm', 'Florida': 'https://www.usachurches.org/church-in-fl.htm', 'Georgia': 'https://www.usachurches.org/church-in-ga.htm', 'Hawaii': 'https://www.usachurches.org/church-in-hi.htm', 'Idaho': 'https://www.usachurches.org/church-in-id.htm', 'Illinois': 'https://www.usachurches.org/church-in-il.htm', 'Indiana': 'https://www.usachurches.org/church-in-in.htm', 'Iowa': 'https://www.usachurches.org/church-in-ia.htm', 'Kansas': 'https://www.usachurches.org/church-in-ks.htm', 'Kentucky': 'https://www.usachurches.org/church-in-k

In [8]:
print(len(states))

51


In [9]:
# del states['Evangelical Lutheran Church in America']
# print(len(states))

In [17]:
def get_cities_urls(states_dict):
    cities_urls = {}

    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'
    }

    for state, url in states_dict.items():
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print(f"Failed. Status code: {response.status_code}")
            continue

        soup = BeautifulSoup(response.content, 'html.parser')

        cities_link = soup.find('a', string=lambda s: s and 'churches by city' in s, href=True)
        if cities_link:
            cities_url = cities_link['href']
            cities_urls[state] = cities_url
        else:
            print(f'No city link found for {state}')

    return cities_urls

In [18]:
cities_urls_dict = get_cities_urls(states)
print(cities_urls_dict)

{'Alabama': 'https://www.usachurches.org/cities-al.htm', 'Alaska': 'https://www.usachurches.org/cities-ak.htm', 'Arizona': 'https://www.usachurches.org/cities-az.htm', 'Arkansas': 'https://www.usachurches.org/cities-ar.htm', 'California': 'https://www.usachurches.org/cities-ca.htm', 'Colorado': 'https://www.usachurches.org/cities-co.htm', 'Connecticut': 'https://www.usachurches.org/cities-ct.htm', 'Delaware': 'https://www.usachurches.org/cities-de.htm', 'Florida': 'https://www.usachurches.org/cities-fl.htm', 'Georgia': 'https://www.usachurches.org/cities-ga.htm', 'Hawaii': 'https://www.usachurches.org/cities-hi.htm', 'Idaho': 'https://www.usachurches.org/cities-id.htm', 'Illinois': 'https://www.usachurches.org/cities-il.htm', 'Indiana': 'https://www.usachurches.org/cities-in.htm', 'Iowa': 'https://www.usachurches.org/cities-ia.htm', 'Kansas': 'https://www.usachurches.org/cities-ks.htm', 'Kentucky': 'https://www.usachurches.org/cities-ky.htm', 'Louisiana': 'https://www.usachurches.org/c

In [24]:
def get_each_city_url(cities_dict):
    each_city_urls = {}

    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'
    }

    for city, url in cities_dict.items():
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print(f"Failed. Status code: {response.status_code}")
            continue

        soup = BeautifulSoup(response.content, 'html.parser')

        city_links = soup.find_all('a', href=lambda href: href and 'search' in href)
        
        if city_links:
            each_city_urls[city] = [link['href'] for link in city_links]
        else:
            print(f'No city links found for {state}')

    return each_city_urls

In [25]:
each_city_urls_dict = get_each_city_url(cities_urls_dict)
print(each_city_urls_dict)

{'Alabama': ['https://www.usachurches.org/search/al/', 'https://www.usachurches.org/search/al/alabaster/', 'https://www.usachurches.org/search/al/alexander-city/', 'https://www.usachurches.org/search/al/andalusia/', 'https://www.usachurches.org/search/al/anniston/', 'https://www.usachurches.org/search/al/ardmore/', 'https://www.usachurches.org/search/al/ashland/', 'https://www.usachurches.org/search/al/athens/', 'https://www.usachurches.org/search/al/atmore/', 'https://www.usachurches.org/search/al/bay-minette/', 'https://www.usachurches.org/search/al/bessemer/', 'https://www.usachurches.org/search/al/birmingham/', 'https://www.usachurches.org/search/al/boaz/', 'https://www.usachurches.org/search/al/centerpoint/', 'https://www.usachurches.org/search/al/columbiana/', 'https://www.usachurches.org/search/al/cottondale/', 'https://www.usachurches.org/search/al/creola/', 'https://www.usachurches.org/search/al/cullman/', 'https://www.usachurches.org/search/al/daleville/', 'https://www.usachu

In [28]:
reducded_url_dict = {state: urls[0] for state, urls in each_city_urls_dict.items()}

In [29]:
reducded_url_dict

{'Alabama': 'https://www.usachurches.org/search/al/',
 'Alaska': 'https://www.usachurches.org/search/ak/',
 'Arizona': 'https://www.usachurches.org/search/az/',
 'Arkansas': 'https://www.usachurches.org/search/ar/',
 'California': 'https://www.usachurches.org/search/ca/',
 'Colorado': 'https://www.usachurches.org/search/co/',
 'Connecticut': 'https://www.usachurches.org/search/ct/',
 'Delaware': 'https://www.usachurches.org/search/de/',
 'Florida': 'https://www.usachurches.org/search/fl/',
 'Georgia': 'https://www.usachurches.org/search/ga/',
 'Hawaii': 'https://www.usachurches.org/search/hi/',
 'Idaho': 'https://www.usachurches.org/search/id/',
 'Illinois': 'https://www.usachurches.org/search/il/',
 'Indiana': 'https://www.usachurches.org/search/in/',
 'Iowa': 'https://www.usachurches.org/search/ia/',
 'Kansas': 'https://www.usachurches.org/search/ks/',
 'Kentucky': 'https://www.usachurches.org/search/ky/',
 'Louisiana': 'https://www.usachurches.org/search/la/',
 'Maine': 'https://www